In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Neural Network

In [16]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

In [17]:
def tanh(Z):
    return np.tanh(Z)

In [18]:
def relu(Z):
    return Z * (Z > 0)

In [ ]:
# Calls specified activation function
def sig(Z,ty):
    if ty == 0:
        return sigmoid(Z)
    if ty == 1:
        return tanh(Z)
    else:
        return relu(Z)

In [ ]:
# Calls specified activation function derivative
def sig_dv(Z,ty):
    if ty == 0:
        return Z*(1-Z)
    if ty == 1:
        return (1-Z*Z)
    else:
        return np.where(Z > 0, 1, 0)

In [19]:
def softmax(A):
    expA = np.exp(A)
    return expA / expA.sum(axis=1,keepdims=True)

In [20]:
# Cost for softmax
def cost(T,Y):
    tot = (-T * np.log(Y))
    return tot.sum()

In [22]:
def predict(Py):
    return np.argmax(Py, axis=1)

In [25]:
def classification_rate(T,Y):
    return np.mean(T == Y)

In [23]:
def forward(M,W,b,actf):
    for i in range(1,len(W)+1):
        if (i != len(W)):
            M[i] = sig((M[i-1].dot(W[i-1]) + b[i-1]), actf)
        else:
            M[i] = M[i-1].dot(W[i-1]) + b[i-1]
            Y = softmax(M[i])
    return Y,M